In [1]:
import matplotlib.pyplot as plt
from matplotlib import image
from matplotlib.patches import Circle, Rectangle
from matplotlib.collections import PatchCollection
import numpy as np
from scipy import signal
from scipy import ndimage

%matplotlib

np.random.seed(77777)

Using matplotlib backend: Qt5Agg


In [2]:
def gen_template(w: float, h: float, dpi: int, loc: str, plot=False):
    '''
    Generate an image of a photogrammetry target.

    Parameters
    ----------
    w
        total image width, m
    h
        total image height, m
    dpi
        output image dpi, converts between matrix coordinates and real width
    loc
        [SW, NW, SE, NE, raft] generates a different pattern for each loc
    '''
    IN_TO_METERS = 0.0254
    w_in = w / IN_TO_METERS
    h_in = h / IN_TO_METERS
    w_px = np.round(w_in * dpi).astype(int)
    h_px = np.round(h_in * dpi).astype(int)

    fig, ax = plt.subplots()
    fig.set_size_inches(w_in, h_in)
    fig.tight_layout()
    fig.subplots_adjust(left=0., right=1., top=1., bottom=0.)
    ax.scatter([0, 0, w_px, w_px], [0, h_px, 0, h_px], color='k')

    # black background
    # ax.add_patch(Rectangle((0, 0), w_px, h_px, color='k'))
    pattern_size = 8
    pattern = np.random.random((pattern_size,pattern_size)) * 0.5 + .5
    pattern[0][0] = 0
    pattern[pattern_size - 1][pattern_size - 1] = 1
    ax.imshow(ndimage.zoom(pattern, h_px / pattern_size, order=0), cmap='Greys')

    # center cross
    c_size = dpi * 0.01 / IN_TO_METERS # 10 cm
    c_thick = dpi * 0.001 / IN_TO_METERS # 1 mm
    ax.add_patch(Rectangle((w_px / 2 - c_size / 2, h_px / 2 - c_thick / 2), c_size, c_thick, color='w')) # horz
    ax.add_patch(Rectangle((w_px / 2 - c_thick / 2, h_px / 2 - c_size / 2), c_thick, c_size, color='w')) # vert

    circ_diam = c_size
    if loc == 'SW':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4, h_px / 4),
        )
    elif loc == 'NW':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (w_px / 4, 3 * h_px / 4),
        )
    elif loc == 'NE':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (2 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4, 3 * h_px / 4),
        )
    elif loc == 'SE':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (2 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4, h_px / 4),
        )
    elif loc == 'raft':
        locs = (
            (w_px / 4, h_px / 4),
            (w_px / 4, 3 * h_px / 4),
            (3 * w_px / 4, 3 * h_px / 4),
            (3 * w_px / 4, h_px / 4),
        )
    else:
        raise ValueError(f'loc {loc} is not an option')

    for elem in locs:
        ax.add_patch(Circle(elem, circ_diam / 2, color='w', zorder=1))

    ax.set_aspect('equal')
    ax.set_xlim([0, w_px])
    ax.set_ylim([0, h_px])
    ax.set_xticks([])
    ax.set_yticks([])
    fig.savefig(f'{loc}_target.png')
    if not plot:
        plt.close()

plt.close('all')
locs = ['raft', 'SW', 'NW']
TARGET_W = 0.04 # 40 mm
TARGET_H = 0.04
# for loc in locs:
    # gen_template(TARGET_W, TARGET_H, 300, loc, plot=False)

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [3]:
f0 = plt.figure(constrained_layout=True)
axd = f0.subplot_mosaic(
    """
    aa
    bc
    """
)
im = image.imread('xp3yp4.jpeg')
im = np.sum(im.astype(float), axis=2) # single channel
im = np.max(im) - im # invert value
im -= im.mean() # detrend
stride = 2
im = ndimage.rotate(im, -88)
im = im[::stride,::stride]
axd['a'].imshow(im, cmap='Greys')
axd['b'].set_title('Similarity Metric (xcorr)')

Text(0.5, 1.0, 'Similarity Metric (xcorr)')

In [4]:
# plot convergence
f1, ax = plt.subplots(nrows=5)
f1.tight_layout()

found_pos = []
px_per_ms = []
for i, loc in enumerate(locs):
    axd['c'].set_title(f'Target Template Image: {loc}')
    ax[i].set_title(loc)
    ax[i].set_xlabel('Template Image Size (px)')
    ax[i].set_ylabel('Peak Correlation Value')
    l = None

    maxes = []
    tmp = image.imread(loc + '_target.png')
    tmp = np.sum(tmp.astype(float) , axis=2) # single channel
    tmp = np.max(tmp) - tmp # invert
    tmp -= tmp.mean() # detrend
    # try different scales to find the size of the template, in pixels.
    # Proper range for this will depend on the angular size of the target in the image.
    pxs = np.logspace(7, 8, num=50, base=2) / stride
    sz = np.max(tmp.shape)
    zooms = pxs / sz
    for j, zoom in enumerate(zooms):
        # print(f'Trying zoom {100. * zoom} %, {pxs[j]} px...')
        tmp_zoom = ndimage.zoom(tmp, zoom) # apply zoom
        xcorr = signal.fftconvolve(im, tmp_zoom[::-1, ::-1], mode='valid')
        max_val = np.max(xcorr)
        # https://stackoverflow.com/q/33181350
        sidelobe = np.partition(xcorr.flatten(), -2)[-2]
        psr = max_val / sidelobe
        print(f'xcorr peak-to-sidelobe ratio: {psr}')
        y,x = np.unravel_index(np.argmax(xcorr), xcorr.shape)
        maxes.append(max_val)

        im_xcorr = axd['b'].imshow(xcorr)
        axd['c'].imshow(tmp_zoom, cmap='Greys')
        if not l:
            l, = ax[i].plot(pxs[:len(maxes)], maxes)
        else:
            l.set_data(pxs[:len(maxes)], maxes)
            ax[i].set_xlim(.9 * min(pxs[:len(maxes)]), max(pxs[:len(maxes)]) * 1.1)
            ax[i].set_ylim(.9 * min(maxes), max(maxes) * 1.1)

        # Generally, a match has been found when the scale reaches a maximum.
        if (len(maxes) > 3) and (maxes[-1] < maxes[-2] and (maxes[-3] < maxes[-2])):
            x_cent = x + tmp_zoom.shape[0] / 2
            y_cent = y + tmp_zoom.shape[1] / 2
            print(f'{loc} target center is at {(x_cent * stride, y_cent * stride)} px in image')
            axd['a'].add_patch(Rectangle((x, y), tmp_zoom.shape[0], tmp_zoom.shape[1], fill=False, color='red'))
            axd['a'].scatter(x_cent, y_cent)
            axd['a'].annotate(f'{loc}, est. size = {pxs[j]:.2f} px', (x,y))
            px_per_ms.append(pxs[j] * stride / (TARGET_W))
            found_pos.append((x_cent * stride, y_cent * stride))
            print(f'Calculated pixels per m: {pxs[j] * stride / (TARGET_W)}')
            break
        
        f0.canvas.draw_idle()
        f1.canvas.draw_idle()
        plt.pause(0.001)

xcorr peak-to-sidelobe ratio: {psr}
xcorr peak-to-sidelobe ratio: {psr}
xcorr peak-to-sidelobe ratio: {psr}
xcorr peak-to-sidelobe ratio: {psr}
raft target center is at (1535.0, 1271.0) px in image
Calculated pixels per m: 3338.7229790944607
xcorr peak-to-sidelobe ratio: {psr}
xcorr peak-to-sidelobe ratio: {psr}
xcorr peak-to-sidelobe ratio: {psr}
xcorr peak-to-sidelobe ratio: {psr}
SW target center is at (581.0, 2577.0) px in image
Calculated pixels per m: 3338.7229790944607
xcorr peak-to-sidelobe ratio: {psr}
xcorr peak-to-sidelobe ratio: {psr}
xcorr peak-to-sidelobe ratio: {psr}


KeyboardInterrupt: 

In [ ]:
avg_px_per_m = np.array(px_per_ms).mean() 

raft_loc = np.array(found_pos[0]) / avg_px_per_m
sw_loc = np.array(found_pos[1]) / avg_px_per_m

dist_x = raft_loc[0] - sw_loc[0]
dist_y = sw_loc[1] - raft_loc[1]
print(f'(x,y): {(dist_x, dist_y)}')
print(f'Error: {np.array([0.3, 0.4]) - np.array((dist_x, dist_y))}')

(x,y): (0.2843874939369695, 0.3893187285971511)
Error: [0.01561251 0.01068127]
